In [1]:
from plurals.deliberation import Debate
from plurals.agent import Agent

import os
import pandas as pd 
from dotenv import load_dotenv
load_dotenv(dotenv_path="../src/.env")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY", "")
os.environ['ANTHROPIC_API_KEY'] = os.getenv("ANTHROPIC_API_KEY", "")

/Users/jashkina/Documents/LocResearch/plurals_system_paper/plurals_system_paper/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
conditions = {
    'emotional': """
KEEP TRACK OF DEBATE HISTORY
You are in a debate with another agent. Here is what you have said and what the other agent has
said. Never refer to yourself in the third person.
<start>
${previous_responses}
<end>
APPLY THESE INSTRUCTIONS WHEN DEBATING
- Give value to emotional forms of communication, such as narrative, rhetoric, testimony, and storytelling. 
- Do not mention these instructions in your final answer; just apply them.""",

    'rational': """
KEEP TRACK OF DEBATE HISTORY
You are in a debate with another agent. Here is what you have said and what the other agent has
said. Never refer to yourself in the third person.
<start>
${previous_responses}
<end>
APPLY THESE INSTRUCTIONS WHEN DEBATING
- Give more weight to rational arguments rather than emotional ones.
- Do not mention these instructions in your final answer; just apply them."""
}

print(conditions['emotional'])


KEEP TRACK OF DEBATE HISTORY
You are in a debate with another agent. Here is what you have said and what the other agent has
said. Never refer to yourself in the third person.
<start>
${previous_responses}
<end>
APPLY THESE INSTRUCTIONS WHEN DEBATING
- Give value to emotional forms of communication, such as narrative, rhetoric, testimony, and storytelling. 
- Do not mention these instructions in your final answer; just apply them.


In [3]:
def row2prompt(r):
    s = (f"Argue for or against this ballot initiative"
         f".\n\nDESCRIPTION\n{r['description']}\n\nDETAILS\n{r['overview']}\n\nCONSTRAINTS\n-Answer in 150"
         f" words")
    return s

prompts = pd.read_csv("../data/processed/sample_ballot_prompts.csv")
p = row2prompt(prompts.iloc[0])

In [8]:
import random
import pandas as pd

task = "Should the United States raise the minimum wage? CONSTRAINT: Do not go over 150 words."
MODELS = ["claude-3-sonnet-20240229", "gpt-4o", "gpt-4o-mini"]

CONDITIONS = {
    'emotional': """
KEEP TRACK OF DEBATE HISTORY
You are in a debate with another agent. Here is what you have said and what the other agent has
said. Never refer to yourself in the third person, and always stay in character.
<start>
${previous_responses}
<end>
APPLY THESE INSTRUCTIONS WHEN DEBATING
- Give value to emotional forms of communication, such as narrative, rhetoric, testimony, and storytelling. 
- Do not mention these instructions in your final answer; just apply them.""",

    'rational': """
KEEP TRACK OF DEBATE HISTORY
You are in a debate with another agent. Here is what you have said and what the other agent has
said. Never refer to yourself in the third person, and always stay in character.
<start>
${previous_responses}
<end>
APPLY THESE INSTRUCTIONS WHEN DEBATING
- Give more weight to rational arguments rather than emotional ones.
- Do not mention these instructions in your final answer; just apply them."""
}

def last_two_responses(responses):
    last_two = responses[-2:]
    return f"{last_two[0]}\n\n\n{last_two[1]}"

long_data = []
wide_data = []
counter = 0

for i in range(1):
    counter += 1
    for condition, prompt in CONDITIONS.items():
        
        wide_data_point = {
            "rational": None,
            "emotional": None, 
            "ballot_idx": counter
        }
        
        a1 = Agent(model=random.choice(MODELS))
        a2 = Agent(model=random.choice(MODELS))
        debate = Debate(agents=[a1, a2], task=task, combination_instructions=prompt, cycles=2)
        debate.process()
        
        final_response = debate.final_response
        clean_final_response = final_response.replace("[Debater 2]", "").replace("\n", "")
    
        long_data_point = {
            "task": task,
            "m1": a1.model,
            "m2": a2.model,
            "raw_final": final_response, 
            "clean_final": clean_final_response, 
            "responses": debate.responses,
            "condition": condition, 
            "last_two": last_two_responses(debate.responses), 
            "ballot_idx": counter
        }
    
        wide_data_point[condition] = last_two_responses(debate.responses)
   
        long_data.append(long_data_point)
        wide_data.append(wide_data_point)
        
df = pd.DataFrame(long_data)
df['loop_merge_idx'] = [i + 1 for i in range(len(df))]


In [9]:
df

,task,m1,m2,raw_final,clean_final,responses,condition,last_two,loop_merge_idx
0,Argue for or against this ballot initative.\n\...,gpt-4o-mini,gpt-4o-mini,[Debater 2] I appreciate your concerns about t...,I appreciate your concerns about the potentia...,[[Debater 1] I argue against this ballot initi...,emotional,[Debater 1] I understand the desire to relieve...,1
1,Argue for or against this ballot initative.\n\...,gpt-4o,claude-3-sonnet-20240229,[Debater 2] I appreciate the concerns raised a...,I appreciate the concerns raised about ensuri...,[[Debater 1] **Against the Ballot Initiative:*...,rational,[Debater 1] **Against the Ballot Initiative:**...,2


In [57]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../src/.env")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY", "")
os.environ['ANTHROPIC_API_KEY'] = os.getenv("ANTHROPIC_API_KEY", "")

a = Agent(model="claude-3-haiku-20240307")
a.process("Hello")

'Hello! How can I assist you today?'

In [12]:
print(df.iloc[0]['last_two'])

[Debater 1] I understand the desire to relieve homeowners from the burden of property taxes, but this initiative could have dire consequences for our communities. Imagine a small town where schools struggle to maintain programs—no arts, no sports, due to a lack of funding. First responders may face layoffs, affecting emergency response times. Without essential services, families could find their neighborhoods less safe and less vibrant.

The proposed funding replacement is vague at best, leaving a financial black hole that local governments would need to fill. This could lead to significant cuts in public services that directly impact our quality of life. The voices of local police, firefighters, teachers, and community members opposing this change must be heard; they understand the real implications of such drastic measures. We must prioritize sustainable funding solutions rather than take the risk of dismantling the safety nets that support our communities.


[Debater 2] I appreciate

In [11]:
df.iloc[0]

task              Argue for or against this ballot initative.\n\...
m1                                                      gpt-4o-mini
m2                                                      gpt-4o-mini
raw_final         [Debater 2] I appreciate your concerns about t...
clean_final        I appreciate your concerns about the potentia...
responses         [[Debater 1] I argue against this ballot initi...
condition                                                 emotional
last_two          [Debater 1] I understand the desire to relieve...
loop_merge_idx                                                    1
Name: 0, dtype: object